In [1]:
# inspired by VPT cleaning process, use model, preprocess = clip.load('RN50x64', device, jit=True)
#!pip install matplotlib
#!pip install -U yt-dlp
#!pip install ffmpeg-python

import yt_dlp
import matplotlib
from yt_dlp import YoutubeDL
import random
import os


def s_to_HHmmss(secs):
    mins = (secs - secs%60) / 60
    secs = secs%60
    hours = (mins - mins%60) / 60
    mins = mins%60
    
    if hours>0:
        HHmmss = ":".join([str(int(hours)),str(int(mins)),str(secs)])
    elif mins>0:
        HHmmss = ":".join([str(int(mins)),str(secs)])
    else:
        HHmmss = str(secs)
            
    return HHmmss

Duplicate key in file PosixPath('/home/idmi/miniconda3/envs/minerl/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 603 ('image.interpolation:\tnone')


In [2]:
# OPEN VIDEO FILE HEADER, READ SEEN LINKS/CHANNELS
header_file = open("spamham_samples/header.txt", "r")
header_text = header_file.readlines()
header_file.close()

all_links = []
all_channels = []

# https://pypi.org/project/yt-dlp/#usage-and-options
def longer_than_a_minute(info, *, incomplete):
    """Download only videos longer than 10 minutes"""
    duration = info.get('duration')
    if duration and duration < 600:
        return 'The video is too short'
YDL_OPTIONS = {'noplaylist':True, 'Download':False, 'match_filter':longer_than_a_minute, 'ignoreerrors':True}

for line in header_text:
    link = line
    if (not (link in all_links)) and (link!=''):
        all_links.append(link)

In [3]:
search_queries = [
#    "minecraft fresh start new world creative",
#    "minecraft new world modded",
#    "minecraft guide",
#    "minecraft speedrun",
#    "minecraft how to redstone",
#    "minecraft new world tutorial",
#    "minecraft creative guide",
#    "minecraft tutorial how to survival vanilla new world lets play",
#    "minecraft basics",
#    "minecraft for beginners",
#    "minecraft how to hardcore guide",
#    "minecraft how to survival 1.16 guide",

    'minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top',
    'minecraft "vanilla" "new world" "survival"',
    'minecraft "vanilla" "new world" "survival" "lets play"',
    'minecraft "vanilla" "new world" "survival" "lets play" "tutorial"',
    'minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top vod',
    'minecraft longplay',
    ''

#    "minecraft stream",
#    "minecraft lets play",
#    "minecraft gameplay survival",
#    "minecraft gameplay hardcore",
#    "minecraft lets play",
#    "minecraft tutorial",
#    "minecraft new world survival",
#    "minecraft hardcore",
#    "minecraft gameplay surival",
#    "minecraft longplay",
    ]

for query in search_queries:

    search_query = query
    
    #"minecraft tutorial how to hardcore vanilla new world lets play"
    # minecraft vanilla lets play stream
    #"minecraft guide walkthrough survival new world 1.16"
    #"minecraft survival hardcore tutorial how to guide learn basics beginner lets play walkthrough gameplay new world first night first day 1.16"
    #"minecraft new world creative vanilla"
    #"minecraft creative mode how to"
    # "minecraft shaders"
    #"minecraft console edition pocket edition ps3 ps4 ps5 xbox bedrock"
    #"playing minecraft with multiplayer server"
    # minecraft creative 1.16
    # minecraft creative
    # minecraft lets play stream mod
    # minecraft mod technic
    # minecraft mod

    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        search_results = ydl.extract_info(f"ytsearchall:{search_query}", download=False)['entries']

    # DOWNLOAD RANDOM FRAMES FROM EACH LINK
    #https://pypi.org/project/yt-dlp/



    # dont re-download frames from videos which have already hadframes downloaded
    with open("spamham_samples/header.txt","r") as used_links:
        used_links = used_links.readlines()
        for i in range(len(used_links)):
            used_links[i] = used_links[i].split('\n')[0]



    # for every link:
    for i in range(len(search_results)):

        
        # separate out video info
        video = search_results[i]
        try:
            video['duration']
        except:
            continue

        if video['duration']>300 and not(video['id'] in used_links):      ############################# PRE-FILTERING FILTERING:

            video_link = video['id']
            video_name = video['title']
            video_channel = video['channel']
            filename = video_link

            print("getting new video!")
            # SELECT RANDOM FRAME
            duration = video['duration']
            random_sec = min(random.randint(0, 10*60), duration-2)
            random_sec_formatted = "*"+random_sec+"-"+random_sec+1


            # DOWNLOAD SHORT VIDEO SECTION
            full_link = str("https://www.youtube.com/watch?v=%s"%video_link)
            print(full_link)
            print(random_sec_formatted)

            filename_ext = filename+'.mp4'
            command_download_frames = str("yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections %s %s --output spamham_samples/%s.mp4" %(random_sec_formatted, full_link, filename))
            #command_get_1_frame = str('ffmpeg -i spamham_samples/%s -vcodec png -ss 0 -vframes 1 -an -f rawvideo spamham_samples/frames/%s.png -y' %(filename_ext, filename) )
            command_get_1_frame = str('ffmpeg -i spamham_samples/%s -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/%s.jpeg -y' %(filename_ext, filename) )
            command_del_video = str('rm spamham_samples/%s.mp4' %filename)
            command_all_in_1 = command_download_frames +' && ' + command_get_1_frame +' && ' + command_del_video
            os.system(command_all_in_1)

            # CROP VIDEO SECTION TO FIRST FRAME
            #"https://www.youtube.com/watch?v={sID}".format(sID=video_link)
            #label = get_response(filename+".jpg")
            line = str(filename+"\n")

            header_file = open("spamham_samples/header.txt","a")
            sources_file = open("spamham_samples/sources.txt","a")
            header_file.write(line)
            sources_file.write(video_channel+","+"https://www.youtube.com/watch?v={sID}".format(sID=video_link)+"\n")
            header_file.close()
            sources_file.close()

[youtube:search] Extracting URL: ytsearchall:minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -luc...nsion -"how to" -top
[download] Downloading playlist: minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top
[youtube:search] query "minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top": Downloading web client config
[youtube:search] query "minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top" page 1: Downloading API JSON
[youtube:search] query "minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dime

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'https://rr3---sn-cn3tc-ajt6.googlevideo.com/videoplayback?expire=1675200443&ei=WzPZY466D5XImLAPzqqn8A4&ip=2a02%3Ac7e%3A4f43%3A8700%3Ada9f%3A4ffa%3A97ed%3A8014&id=o-AMDkUMb184XABy4DVY0y57EFnxGjlMHMBzYg-1AEHETh&itag=136&source=youtube&requiressl=yes&mh=w4&mm=31%2C29&mn=sn-cn3tc-ajt6%2Csn-aigl6nzl&ms=au%2Crdu&mv=m&mvi=3&pl=42&initcwndbps=2057500&spc=H3gIhl6YID7PKs1MdZvzTEI6WV7PPG8&vprv=1&svpuc=1&mime=video%2Fmp4&gir=yes&clen=797519581&dur=4462.416&lmt=1649422602698444&mt=1675178232&fvip=3&keepalive=yes&fexp=24007246&c=ANDROID&txp=6216224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRAIgZyYkC6E1yFAL4tLR_yuUVr9S91o9Gf1TDzSbkVRb96ACIGkHoyv57qrvO5XzAz59M4Abo26_CTzdRraFz22oqp8c&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRgIhAOo8SFGjjLBwTMJTLRyRxM20g0Mvhl5fD3QafOxnWEZjAiEA86x-m0E2g-QBEj9g1gBwjvGbdDymNeLd7A9qXbYUDxo%3D':
  Metadata:
    major_brand   

[download] 100% of  343.33KiB in 00:00:07 at 48.29KiB/s
[VideoConvertor] Not converting media file "spamham_samples/L0jkMfnVGfw.mp4"; already is in target format mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

getting new video!
https://www.youtube.com/watch?v=8oa64DG6naM
*5:34-5:35
[youtube] Extracting URL: https://www.youtube.com/watch?v=8oa64DG6naM
[youtube] 8oa64DG6naM: Downloading webpage
[youtube] 8oa64DG6naM: Downloading android player API JSON
[youtube] 8oa64DG6naM: Downloading MPD manifest
[info] 8oa64DG6naM: Downloading 1 format(s): 136+140
[info] 8oa64DG6naM: Downloading 1 time ranges: 334.0-335.0
[download] Destination: spamham_samples/8oa64DG6naM.mp4


...

getting new video!
https://www.youtube.com/watch?v=YpX6WWNefVY
*8:14-8:15
[youtube] Extracting URL: https://www.youtube.com/watch?v=YpX6WWNefVY
[youtube] YpX6WWNefVY: Downloading webpage
[youtube] YpX6WWNefVY: Downloading android player API JSON
[youtube] YpX6WWNefVY: Downloading MPD manifest
[info] YpX6WWNefVY: Downloading 1 format(s): 18
[info] YpX6WWNefVY: Downloading 1 time ranges: 494.0-495.0
[download] Destination: spamham_samples/YpX6WWNefVY.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'https://rr5---sn-cn3tc-ajtee.googlevideo.com/videoplayback?expire=1675202130&ei=8jnZY_zyA-aDxgLI3JaQDw&ip=2a02%3Ac7e%3A4f43%3A8700%3Ada9f%3A4ffa%3A97ed%3A8014&id=o-AOOczAW3qk_Ny22BuxlFFzGmxe8rlkpF4M-Nlq_l3rtA&itag=18&source=youtube&requiressl=yes&mh=FK&mm=31%2C29&mn=sn-cn3tc-ajtee%2Csn-aigl6ned&ms=au%2Crdu&mv=m&mvi=5&pl=42&initcwndbps=1853750&spc=H3gIhgwnq5EGUJ27IEWJ7NChvSO-NLk&vprv=1&svpuc=1&mime=video%2Fmp4&cnr=14&ratebypass=yes&dur=899.494&lmt=1635584392777631&mt=1675180148&fvip=5&fexp=24007246&c=ANDROID&txp=1218224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Ccnr%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRAIgRieKgHRh0ZcxvbP9dJvIll5JqVwuhAm2Q-1Dwv7SdQcCIE9tn0uFI88RV8yTPpXyLoVwWeTubhSz2h1mPI_7K4_M&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRAIgJtOJ-SWNuZEmOgMdVCZ2C-Raupn9nuHageW4kVzqnosCIB4tmZY2rfqI1Jg4ytKBOR1eAhW1lI4EUMHPsfLb3Jfg':
  Metadata:
    major_brand     : mp42
    min

[download] 100% of  150.26KiB in 00:00:03 at 40.04KiB/s
[VideoConvertor] Not converting media file "spamham_samples/YpX6WWNefVY.mp4"; already is in target format mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

getting new video!
https://www.youtube.com/watch?v=1B1oXSHaOTI
*1:58-1:59
[youtube] Extracting URL: https://www.youtube.com/watch?v=1B1oXSHaOTI
[youtube] 1B1oXSHaOTI: Downloading webpage
[youtube] 1B1oXSHaOTI: Downloading android player API JSON
[youtube] 1B1oXSHaOTI: Downloading MPD manifest
[info] 1B1oXSHaOTI: Downloading 1 format(s): 136+140
[info] 1B1oXSHaOTI: Downloading 1 time ranges: 118.0-119.0
[download] Destination: spamham_samples/1B1oXSHaOTI.mp4
